In [3]:
import pandas as pd

splits = {'train': 'data/train-00000-of-00001.parquet', 'validation': 'data/validation-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/allenai/sciq/" + splits["train"])
df

,question,distractor3,distractor1,distractor2,correct_answer,support
0,What type of organism is commonly used in prep...,viruses,protozoa,gymnosperms,mesophilic organisms,"Mesophiles grow best in moderate temperature, ..."
1,What phenomenon makes global winds blow northe...,tropical effect,muon effect,centrifugal effect,coriolis effect,Without Coriolis Effect the global winds would...
2,Changes from a less-ordered state to a more-or...,endothermic,unbalanced,reactive,exothermic,Summary Changes of state are examples of phase...
3,What is the least dangerous radioactive decay?,zeta decay,beta decay,gamma decay,alpha decay,All radioactive decay is dangerous to living t...
4,Kilauea in hawaii is the world’s most continuo...,magma,greenhouse gases,carbon and smog,smoke and ash,Example 3.5 Calculating Projectile Motion: Hot...
...,...,...,...,...,...,...
11674,The enzyme pepsin plays an important role in t...,lipids,protons,proteins,peptides,Protein A large part of protein digestion take...
11675,What remains a constant of radioactive substan...,acidity,temperature,volatility,rate of decay,The rate of decay of a radioactive substance i...
11676,"Terrestrial ecosystems, also known for their d...",substrates,bisomes,monomes,biomes,"Terrestrial ecosystems, also known for their d..."
11677,High explosives create shock waves that exceed...,turbulence,light speed,ion speed,supersonic,The modern day formulation of gun powder is ca...


In [16]:
df.loc[0]

question          What type of organism is commonly used in prep...
distractor3                                                 viruses
distractor1                                                protozoa
distractor2                                             gymnosperms
correct_answer                                 mesophilic organisms
support           Mesophiles grow best in moderate temperature, ...
Name: 0, dtype: object

In [9]:
import torch
import os

from sentence_transformers import CrossEncoder
from src.chain_initialisation import init_chain
from src.embedding_management import init_embeddings, add_embeddings_from_files
from src.model_initialisation import init_model
from src.pipeline_initialisation import init_pipeline
from src.retriever_initialisation import init_retriever
from src.dynamic_doc_retrieval import download_documents, initialise_keyword_model, generate_query_from_question

In [8]:
cuda_available = torch.cuda.is_available()
print(f"Initializing model... CUDA available: {cuda_available}")
model, model_name = init_model(cuda_available)
pipeline = init_pipeline(model=model, model_name=model_name)
vectorstore = init_embeddings(cuda_available)
retriever = init_retriever(vectorstore)
reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
chain = init_chain(pipeline=pipeline, retriever=retriever)
keyword_model = initialise_keyword_model()
print("Start-up complete.")

Initializing model... CUDA available: False


Device set to use cpu
/home/henri/ul-fri-nlp-course-project-2024-2025-drop-table-teams/src/embedding_management.py:17: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(


Loading FAISS from file - skipping embedding build.
Vectorstore loaded successfully.
Retriever initialized with k=2 and similarity threshold=0.8


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

/home/henri/ul-fri-nlp-course-project-2024-2025-drop-table-teams/src/chain_initialisation.py:7: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipeline)


LLM model chain created.
Start-up complete.


In [12]:
BASE_DIR = os.curdir
DATA_DIR = os.path.join(BASE_DIR, "data")
DYNAMIC_DATA_DIR = os.path.join(DATA_DIR, "dynamic")
print(f"dynamic data dir: {DYNAMIC_DATA_DIR}")

def ask_question(question, online=True):
    if online:
        print("Obtaining data from scholar...")
        query = generate_query_from_question(keyword_model, question)
        num_docs: int = int(os.getenv("num_docs"))
        max_tries: int = int(os.getenv("max_tries"))
        downloaded_files = download_documents(query, DYNAMIC_DATA_DIR, num_docs, max_tries)
        add_embeddings_from_files(vectorstore, downloaded_files)

    response = chain.invoke({
        "question": question,
        "chat_history": []
    })
    answer = response["answer"].split("### Answer:")[-1].strip()
    source_documents = response["source_documents"]
    return answer, source_documents

dynamic data dir: ./data/dynamic


In [13]:
question = df.loc[0]["question"]
answer, source_docs = ask_question(question, online=False)
answer

/home/henri/.local/lib/python3.10/site-packages/langchain_core/vectorstores/base.py:1082: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='7e08dabe-c118-4506-8c9e-d911c1835182', metadata={'source': '/home/henri/ul-fri-nlp-course-project-2024-2025-drop-table-teams/data/fao780/fao780/docsutf8/t5816e.txt'}, page_content='A list of species produced in the region is given in Table 5.\n2.3 Production systems and practices in the region\nA diversity of species is cultured throughout the region in relatively small volumes. Consequently a wide variety of aquaculture practices are used, ranging from intensive and semi-intensive systems for producing salmonids and marine shrimp, to extensive systems for the culture of fish, giant clams, seaweed, and pearl oysters.'), np.float32(-24.619614)), (Document(id='2ee5e358-a0ea-4f13-b6d0-56c34630a209', metadata={'source': '/home/henri/ul-fri-nlp-course-project-2024-2025-drop-table-teams/data/fao780/fao780/docsutf8/x0209e.txt'}, pa

"The following two animals-- fish (from which milk may also come) and other fishes--- include an invertebrate named Megaptera klassensis although its genus does not exist at present there.(Tasmanian Sea Turtles.) These species live around 70 percent of the ocean basins covering about 65 percent water(Suez Formation Water Source). Aquamarine waters typically contain less than five feet between three degrees south latitude=0 deg C..ocean surface temperature< 30 °C > below sea level range on both ends of the main boundary line during estuaries up to 4° s(-1 ) above 100 yz...[p]atoll ice thickness < 0.5 mm−2; maximum energy needed per unit volume ~ 20 Mg((m*r+8)/4 ); most bodies form into four'submersible' submaploidic marine mounds 2 km² apart including cetaceans plus halos.[i]The vast majority rely upon saltwater derived material after several thousand years [and more significantly provide dietary nutrients]. In some instances their diets consist primarily comprised sugar based meals con

In [15]:
from sentence_transformers import SentenceTransformer, util
from fuzzywuzzy import fuzz

# Initialize embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

In [23]:
def evaluate_answer(row):
    question = row['question']
    correct_answer = row['correct_answer']
    support = row['support']
    distractors = [row['distractor1'], row['distractor2'], row['distractor3']]

    # Chatbot answer
    response, source_docs = ask_question(question)
    response = response.lower()

    # Exact match
    exact = int(correct_answer.lower() in response)

    # Fuzzy match
    fuzzy_score = fuzz.partial_ratio(correct_answer.lower(), response)

    # Embedding similarity to correct answer
    emb_correct = model.encode(correct_answer, convert_to_tensor=True)
    emb_response = model.encode(response, convert_to_tensor=True)
    sim_correct = util.cos_sim(emb_correct, emb_response).item()

    # Embedding similarity to support passage
    emb_support = model.encode(support, convert_to_tensor=True)
    sim_support = util.cos_sim(emb_support, emb_response).item()

    # Compare similarity to distractors
    sim_distractors = []
    for d in distractors:
        emb_distractor = model.encode(d, convert_to_tensor=True)
        sim_distractors.append(util.cos_sim(emb_distractor, emb_response).item())
    
    # Is chatbot closer to correct than to all distractors?
    correct_beats_distractors = int(sim_correct > max(sim_distractors))

    return {
        "question": question,
        "chatbot_response": response,
        "exact_match": exact,
        "fuzzy_score": fuzzy_score,
        "sim_to_correct": round(sim_correct, 3),
        "sim_to_support": round(sim_support, 3),
        "correct_beats_distractors": correct_beats_distractors,
        "source_docs": source_docs
    }


In [24]:
df.loc[:3]

,question,distractor3,distractor1,distractor2,correct_answer,support
0,What type of organism is commonly used in prep...,viruses,protozoa,gymnosperms,mesophilic organisms,"Mesophiles grow best in moderate temperature, ..."
1,What phenomenon makes global winds blow northe...,tropical effect,muon effect,centrifugal effect,coriolis effect,Without Coriolis Effect the global winds would...
2,Changes from a less-ordered state to a more-or...,endothermic,unbalanced,reactive,exothermic,Summary Changes of state are examples of phase...
3,What is the least dangerous radioactive decay?,zeta decay,beta decay,gamma decay,alpha decay,All radioactive decay is dangerous to living t...


In [25]:
results = [evaluate_answer(row) for _, row in df.loc[:3].iterrows()]
results_df = pd.DataFrame(results)
results_df

Obtaining data from scholar...
Querying Google Scholar for: yogurt foods cheese organism preparation

Result 1: Potential effects of probiotics in cheese and yogurt production: A review
Publication URL: https://analyticalsciencejournals.onlinelibrary.wiley.com/doi/abs/10.1002/elsc.201100122
Trying to download PDF from: https://www.academia.edu/download/107457162/elsc.20110012220231117-1-j22l0l.pdf


/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.academia.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.academia.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


PDF saved as ./data/dynamic/Potential_effects_of_probiotics_in_cheese_and_yogurt_production_A_review.pdf

Result 2: Inhibitory effects of Microgard™ on yogurt and cottage cheese spoilage organisms
Publication URL: https://www.sciencedirect.com/science/article/pii/S0022030290787449
Trying to download PDF from: https://www.sciencedirect.com/science/article/pii/S0022030290787449/pdf?md5=271e01ddf92d493dc06ad431140c96c9&pid=1-s2.0-S0022030290787449-main.pdf


/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.sciencedirect.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
[INFO] | 2025/05/28 12:59:25 | Choose scihub url [0]: https://sci-hub.se
[INFO] | 2025/05/28 12:59:25 | <- Request: scihub_url=https://sci-hub.se, source=DoiSource[type=doi, id=www.sciencedirect.com/science/article/pii/S0022030290787449], proxies={'http': 'socks5://127.0.0.1:7890'}


The eprint URL is not a direct link to a PDF.


...

PDF saved as ./data/dynamic/Ionizing_Radiation_Hazards_Dangerous_Goods_IV.pdf


/home/henri/.local/lib/python3.10/site-packages/langchain_core/vectorstores/base.py:1082: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='e777ad1d-2e2d-4a62-a543-d377323334d7', metadata={'source': './data/dynamic/Modeling_radioactive_decay.pdf'}, page_content='Keywords : Half-mean life, modeling, radioactive decay, radioactivity teaching, simulation \nI. Introduction  \nRadioactive decay is the name given to the natural process by which an atomic nucleus spontaneously transforms \nitself (decays) into another, releasing dangerous high energy emissions (Hughes and Zalts, 2000). \nIf a sample consists of N0 radioactive nuclei at time t=0, and if N(t) denotes the number of the nuclei remaining at'), np.float32(-18.21845)), (Document(id='3ebdfe23-d04b-4e11-876b-8a55d844675b', metadata={'source': './data/dynamic/Modes_of_radioactive_decay.pdf'}, page_content='Particles released in radioactive nuclear decay\n(1)\nα\nα particle (helium nucleus)\n(2)\nβ−\nElectron e−a

Vectorstore updated with 630 document splits.


This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (1024). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


IndexError: index out of range in self